# Thông tin nhóm
1612406 - Đặng Phương Nam

1612423 - Lê Minh Nghĩa

# Thu thập dữ liệu từ API
- Sử dụng API Dark Sky: để thu thập dữ liệu thời tiết của TP. Hồ Chí Minh (https://darksky.net/forecast/10.75,106.6667/us12/en)
- Đăng ký tài khoản dev để nhận khóa API tại https://darksky.net/dev
- API Dark chỉ cho phép tối đa thực hiện 1000 request mỗi ngày đối với dùng free,còn muốn thực hiện nhiều hơn thì phải trả phí ($0,0001 cho mỗi ngày).

url: https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
+ Với [key] là secret key mà Dark Sky cung cấp cho mỗi tài khoản dev đã đăng ký ở trên, ví dụ  "b6b7262c996a461fba2fbd016457c40d" (khi nào dùng xong nhóm sẽ reset lại secret key).
+ [latitude],[longitude] lần lượt là kinh độ và vĩ độ tương ứng của địa điểm mà mình muốn request. Ở đây tọa độ TP Hồ Chí Minh tương ứng là "10.75,106.6667".
+ time là thời gian muốn lấy dữ liệu thời tiết, dạng UNIX time.

## Yêu cầu lấy 10.000 dữ liệu thời tiết của TP.HCM

In [1]:
# Inport thư viện cần thiết để tính toán thời gian để bắt đầu request
from datetime import datetime, timedelta
import time

### Nếu lấy theo ngày thì tốn 10.000 ngày, do đó tốn đến 10.000 lần request (tốn nhiều nhất 10 ngày liên tục)

In [2]:
# YYYY/MM/DD/Hour/Minute/Sec
date_end = datetime(2019, 11, 30)
date_start = date_end - timedelta(days=10000)
print("Request with 10000 consecutive days:")
print("- Date Start request: ", date_start, "-> Unix Timestamp: ", time.mktime(date_start.timetuple()))
print("- Date End request:   ", date_end, "-> Unix Timestamp: ", time.mktime(date_end.timetuple()))

Request with 10000 consecutive days:
- Date Start request:  1992-07-14 00:00:00 -> Unix Timestamp:  711046800.0
- Date End request:    2019-11-30 00:00:00 -> Unix Timestamp:  1575046800.0


Vậy thôi lấy date_start là 1992-07-01 luôn đi, rồi cộng dồn từ từ cho đủ 10.000 ngày.

#### Dữ liệu mấy ngày gần đây có dạng: nhiều feature hơn

{"time":1574704800,"summary":"Humid and Partly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":79.3,"apparentTemperature":84.08,"dewPoint":77.17,"humidity":0.93,"pressure":1011.4,"windSpeed":4.41,"windGust":4.71,"windBearing":42,"cloudCover":0.44,"uvIndex":0,"visibility":4.974,"ozone":251.3}

#### Về mấy ngày xa xôi trước đó thì có dạng: ít feature hơn

{"time":710618400,"temperature":74.88,"apparentTemperature":76.36,"dewPoint":72.14,"humidity":0.91,"pressure":1008,"windSpeed":0,"cloudCover":0.88,"uvIndex":0}

### Nếu lấy theo giờ thì tốn 10.000 giờ (lưu ý mỗi ngày sẽ thu được 24 data weather ứng với 24 tiếng), vậy chỉ tốn khoảng 417 lần request (chỉ 1 ngày là xong)

In [3]:
# YYYY/MM/DD/Hour/Minute/Sec
date_end = datetime(2019, 11, 30)
date_start = date_end - timedelta(hours=10000)
print("Request with 10000 consecutive days:")
print("- Date Start request: ", date_start, "-> Unix Timestamp: ", time.mktime(date_start.timetuple()))
print("- Date End request:   ", date_end, "-> Unix Timestamp: ", time.mktime(date_end.timetuple()))

Request with 10000 consecutive days:
- Date Start request:  2018-10-09 08:00:00 -> Unix Timestamp:  1539046800.0
- Date End request:    2019-11-30 00:00:00 -> Unix Timestamp:  1575046800.0


Vậy thôi lấy date_start là 2018-10-09 00:00:00 luôn đi, rồi cộng dồn từ từ cho đủ 10.000 giờ.

#### Dữ liệu từ năm 2018 đến 2019 thì chắc có đầy đủ các feature giống nhau rồi.

### - Vậy lúc lấy theo ngày thì liệu có đủ feature để huấn luyện hay không?
### - Còn lấy theo giờ thì có phù hợp với bài toán của mình hay không? (Do định dùng dữ liệu của 3 hay 5 ngày trước đó để dự đoán nhiệt độ hay thời tiết (mưa hay nắng) của ngày tiếp theo.
### -  Xét về thời gian request thì cái lấy theo giờ nhanh hơn nhiều (chỉ tốn 1 ngày).

## Tiến hành request (sau khi đã chọn được thời gian lấy)

In [ ]:
import requests
import json

key = "b6b7262c996a461fba2fbd016457c40d" # secret key
latitude = "10.75" # kinh độ TP.HCM
longitude = "106.6667" # vĩ độ TP.HCM
time = "" #Nhớ là Unix Timestamp

records = []
while True:
    try:
        url = f'https://api.darksky.net/forecast/{key}/{latitude},{longitude},{time}'
        print(f'GET {url}')
        r = requests.get(url)
        json_pydata = json.loads(r.text) # Shortcut: json_pydata = r.json()
        
        # Lấy records tương ứng với mấy cái feature mà mình muốn lấy 
        #....
        
        # Cộng time lên theo ngày hay theo giờ?
        #time +=
        
    except Exception as e: # Có thể là do hết lượng request được phép, hoặc hết page, ...
        print(f'Error: {e}')
        break

Xong nhớ lưu lại dùm cái!

In [1]:
import logging
from datetime import datetime

logger = logging.getLogger('crawler')
fhandler = logging.FileHandler(filename=f'{datetime.today().strftime("%Y-%m-%d_%H_%M_%S")}.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.handlers = []
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [2]:
import requests
import json
from datetime import datetime, timedelta
import os.path

#API_KEY = "b6b7262c996a461fba2fbd016457c40d"   # NAM 1
API_KEY = "9d3e4e322240bc68d548ec13b7e3e445"   # NAM 2
#API_KEY = "57c62541be2d79ea4d55fd39e5080dc0" # Nghia
LAT = 10.75
LNG = 106.6667

date_end = datetime(2011, 9, 25) #2011-09-25
n = 10000

fname = f"data/{date_end.strftime('%Y-%m-%d')}.json"
if not os.path.exists(fname):
    s = requests.Session()
    with open(fname, "a") as f:
        for dayDelta in range(n):
            dt = date_end - timedelta(days=dayDelta)
            timestr = dt.strftime("%Y-%m-%dT%H:%M:%S")
            url = f"https://api.darksky.net/forecast/{API_KEY}/{LAT},{LNG},{timestr}?exclude=currently&units=si"
            logger.info(f"GET {url}")
            try: 
                r = s.get(url)
                r.raise_for_status()
                f.write(f'{r.text}')
            except Exception as e:
                logger.error(f"{r.text}\n{e}")
                break
else:
    print("ERROR FILE EXISTS")